# Step 1 - FARS Data Extraction & Preprocessing

Remember to download the raw data files from https://www.nhtsa.gov/file-downloads?p=nhtsa/downloads/FARS/2023/National/
- On the website, download *FARS2023NationalCSV.zip*
- Add the following files to *Dataset* folder: vehicle, person, accident

-------------------------

This is the code for the first step of our pipeline.
In summary:
* Join the 3 files data (Person, Vehicle, Accident), with the greatest granularity possible (based in person)
* Delete duplicate features
* Replace the "Not reported" or "unknown" codes of the features with null values

Missing code handling was a particular challenge as there is no universal value for labelling the "unknown" or "not reported" values. Some features detect the "unknown" as 9999, others as 9, and others as "". This caused the value replacement to be more complex, as we could not take a general approach. For example, we could not replace all 9 or 99 with nulls because it could be the case that a driver was 99 years old or a 9 year old was injured, and, by replacing this value, information was going to be lost. Therefore, the FARS User Manual was reviewed and the following code was developed in order to handle this situation from a better perspective.

A limitation of this approach is that, as some specific missing values were mapped, the code only works on the most recent years of the dataset. In last editions of the database (Ex. before 2008), many features had a different code for missing values.

### How to run the code:

1) Run libraries
2) Run all the sections in order (top to bottom)
3) Run the Use
4) Optional: review the code of each section

In [1]:
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import polars as pl

### Configuration for the field-specific missing codes

Specific missing codes and known numeric columns in FARS.

##### FARS field-specific missing codes (based on FARS User Manual)

In [2]:
# FARS field-specific missing codes (based on FARS User Manual)

FIELD_SPECIFIC_MISSING = {
    
    # Demographics
    "AGE": [998, 999],  # Here the example of the introduction can be seen: 9, 88, 98, 99 are possible ages, so they are not included
    "SEX": [8, 9],
    "RACE": [97, 98, 99],  # Discontinued in 2018
    "HISPANIC": [8, 9, 98, 99],  # 0 as no Fatality, in case other type of analysis wants to be made
    
    # Temporal
    "HOUR": [99],
    "MINUTE": [99],
    "DAY": [99],     # Changed in 2010, but kept in case some raw data was incorrectly labeled
    "MONTH": [99],   # Changed in 2009, but kept in case some raw data was incorrectly labeled
    "DAY_WEEK": [9], # Changed in 2010, but kept in case some raw data was incorrectly labeled
    
    # Injury/Outcome
    "INJ_SEV": [9],
    "DEATH_HR": [88, 99],
    "DEATH_MN": [88, 99],
    
    # Behaviors
    "DRINKING": [8, 9, 98, 99],
    "DRUGS": [8,9],
    "DRUG_DET": [8],
    "SPEEDREL": [8, 9],
    "REST_USE": [20, 96, 98, 99],
    "REST_MIS": [7, 8],
    "AIR_BAG": [98, 99],
    "EJECTION": [7, 8, 9],
    "EJ_PATH": [9],
    "EXTRICAT": [9],
    
    # Vehicle
    "BODY_TYP": [98, 99],
    "MOD_YEAR": [9998, 9999],
    "TOW_VEH": [9],
    "J_KNIFE": [9],
    "MCARR_I1": [0, 77, 99],
    "MCARR_I2": [000000000, 777777777, 999999999],
    "GVWR": [0, 8, 9],   # Discontinued in 2019
    "V_CONFIG": [0, 98, 99],
    "CARGO_BT": [0, 28, 98, 99],
    "HAZ_PLAC": [0, 8],
    "HAZ_ID": [0, 8888],
    "HAZ_CNO": [0, 88],
    "HAZ_REL": [0, 8],
    "BUS_USE": [98, 99],
    "SPEC_USE": [99],
    "EMER_USE": [0, 8, 9],
    "ROLLOVER": [8],
    "ROLINLOC": [8, 9],
    "IMPACT1": [98, 99],
    "FIRE_EXP": [9],
    
    # Environmental
    "WEATHER": [98, 99],
    "WEATHER1": [98, 99],  # Discontinued in 2019
    "WEATHER2": [98, 99],  # Discontinued in 2019
    "LGT_COND": [8,9],
    
    # Location
    "ROUTE": [99],
    "RUR_URB": [8, 9],
    "FUNC_SYS": [98, 99],
    "RD_OWNER": [98, 99],
    "MILEPT": [99998, 99999],
    "LATITUDE": [77.7777000, 88.8888000, 99.9999000],
    "LONGITUD": [777.7777000, 888.8888000, 999.9999000],
    
    # Roadway
    "SP_JUR": [9],
    "HARM_EV": [98, 99],
    "MAN_COLL": [98, 99],
    "RELJCT1": [8, 9],
    "RELJCT2": [98, 99],
    "TYP_INT": [98, 99],
    "WRK_ZONE": [9],
    "REL_ROAD": [98, 99],
    "NOT_HOUR": [88, 99],
    "NOT_MIN": [88, 98, 99],
    "ARR_HOUR": [88, 99],
    "ARR_MIN": [88, 98, 99],
    "HOSP_HR": [88, 99],
    "HOSP_MN": [88, 98, 99],
    
    # Person-specific
    "PER_TYP": [19],
    "SEAT_POS": [98, 99],
    "SEATING": [98, 99],
    "HELM_USE": [20, 98, 99],
    "HELM_MIS": [7, 8],
    "DRUGRES1": [95, 999],  # Discontinued in 2017
    "DRUGRES2": [95, 999],  # Discontinued in 2017
    "DRUGRES3": [95, 999],  # Discontinued in 2017
    "DSTATUS": [8, 9],
    
    # Crash characteristics
    "CF1": [98, 99],
    "CF2": [98, 99],
    "CF3": [98, 99],
    "VTRAFWAY": [8, 9],
    "VNUM_LAN": [8, 9],
    "VSPD_LIM": [98, 99],
    "VALIGN": [9],
    "VPROFILE": [8, 9],
    "VPAVETYP": [8, 9],
    "VSURCOND": [98, 99],
    "VTRAFCON": [97, 99],
    "VTCONT_F": [8, 9],
    
    # Default codes (be aware that there are many features that contain 8,9 as missing values)
    # In the case of this project, due to its scope and purpose, the above features were confirmed to have a correct code replacement
    # In case of a full variables consideration, all should be checked manually
    "_DEFAULT": [88, 98, 99, 997, 998, 999, 9999]
}





##### Numeric columns (common)

In [3]:
# Default numeric columns if not specified
NUMERIC_COLUMNS = [
    # IDs
    "ST_CASE", "VEH_NO", "PER_NO", "STATE", "COUNTY",
    
    # Demographics
    "AGE", "SEX", "RACE", "HISPANIC",
    
    # Temporal
    "YEAR", "MONTH", "DAY", "DAY_WEEK", "HOUR", "MINUTE",
    
    # Outcomes
    "INJ_SEV", "DEATH_HR", "DEATH_MN", "FATALS", "PERSONS",
    
    # Person behavior
    "PER_TYP", "SEAT_POS", "REST_USE", "REST_MIS", "AIR_BAG",
    "EJECTION", "EJ_PATH", "EXTRICAT", "DRINKING", "ALC_RES",
    "ALC_STATUS", "ATST_TYP", "ALC_DET", "DRUGS", "DRUG_DET",
    "DSTATUS", "HOSPITAL", "DOA",
    
    # Vehicle
    "BODY_TYP", "MOD_YEAR", "TOW_VEH", "SPEC_USE", "EMER_USE",
    "ROLLOVER", "ROLINLOC", "IMPACT1", "FIRE_EXP", "SPEEDREL",
    "DR_DRINK", "DR_SF",
    
    # Environmental
    "WEATHER", "WEATHER1", "WEATHER2", "LGT_COND", "SCH_BUS",
    
    # Location
    "ROUTE", "RUR_URB", "FUNC_SYS", "RD_OWNER",
    "MILEPT", "LATITUDE", "LONGITUD", "SP_JUR",
    
    # Crash
    "HARM_EV", "MAN_COLL", "RELJCT1", "RELJCT2", "TYP_INT",
    "WRK_ZONE", "REL_ROAD", "NOT_HOUR", "NOT_MIN", "ARR_HOUR",
    "ARR_MIN", "HOSP_HR", "HOSP_MN", "CF1", "CF2", "CF3"
]

### Step 1.1: Data Loading
The original FARS CSV files are loaded with the columns as strings to prevent conversion issues.

In [4]:
def load_fars_csv(filename: str, data_dir: Path) -> pl.DataFrame:
    path = Path(data_dir) / filename
    
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    
    # Load with no schema inference
    df = pl.read_csv(path, infer_schema_length=0)
    
    print(f"Loaded: {filename}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    return df

### Step 1.2: Key Validation

Remove duplicates and review null keys

In [5]:
def remove_duplicate_keys(
    df: pl.DataFrame,
    key_columns: List[str]
) -> pl.DataFrame:
    # Check for NULL keys
    for col in key_columns:
        if col not in df.columns:
            continue
        
        null_count = df.filter(pl.col(col).is_null()).height
        if null_count > 0:
            print(f"NULL values in {col}: {null_count:,}")
    
    # Remove duplicates
    original_count = df.height
    df_unique = df.unique(subset=key_columns, maintain_order=True)
    duplicates = original_count - df_unique.height
    
    if duplicates > 0:
        print(f"Removed {duplicates:,} duplicate rows")
    else: 
        print(f"No duplicates found")
    
    return df_unique

### Step 1.3: Data Type Conversion

Convert the specified columns from string to numeric

In [6]:
def convert_to_numeric(
    df: pl.DataFrame,
    columns: List[str]
) -> pl.DataFrame:

    converted = 0
    errors = 0
    
    for col in columns:
        if col not in df.columns:
            continue
        
        # Skip if already numeric
        if df[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                             pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                             pl.Float32, pl.Float64]:
            converted += 1
            continue
        
        try:
            df = df.with_columns(
                pl.col(col).cast(pl.Int32, strict=False).alias(col)
            )
            converted += 1
        except Exception as e:
            errors += 1
    
    print(f"Converted {converted} columns to numeric")
    if errors > 0:
        print(f"Errors in {errors} columns")
    
    return df

### Step 1.4: Missing Code Handling
Use the field-specific missing codes dictionary to apply the missing code handling

In [7]:
def replace_field_specific_missing_codes(df: pl.DataFrame) -> pl.DataFrame:
    total_replaced = 0
    columns_modified = 0
    
    for col in df.columns:
        # Get field-specific missing codes
        if col in FIELD_SPECIFIC_MISSING:
            missing_codes = FIELD_SPECIFIC_MISSING[col]
        else:
            # Skip columns not in numeric list and not explicitly defined
            if col not in NUMERIC_COLUMNS:
                continue
            missing_codes = FIELD_SPECIFIC_MISSING["_DEFAULT"]
        
        try:
            col_dtype = df[col].dtype
            
            if col_dtype == pl.Utf8:
                # String column
                missing_code_strs = [str(c) for c in missing_codes if isinstance(c, int)]
                if "" in missing_codes:
                    missing_code_strs.append("")
                
                count_before = df.filter(pl.col(col).is_in(missing_code_strs)).height
                
                if count_before > 0:
                    df = df.with_columns(
                        pl.when(pl.col(col).is_in(missing_code_strs))
                        .then(None)
                        .otherwise(pl.col(col))
                        .alias(col)
                    )
                    total_replaced += count_before
                    columns_modified += 1
            
            elif col_dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                               pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                               pl.Float32, pl.Float64]:
                # Numeric column
                numeric_codes = [c for c in missing_codes if isinstance(c, (int, float))]
                
                count_before = df.filter(pl.col(col).is_in(numeric_codes)).height
                
                if count_before > 0:
                    df = df.with_columns(
                        pl.when(pl.col(col).is_in(numeric_codes))
                        .then(None)
                        .otherwise(pl.col(col))
                        .alias(col)
                    )
                    total_replaced += count_before
                    columns_modified += 1
        
        except Exception:
            continue
    
    print(f" Replaced missing codes in {columns_modified} columns")
    print(f" Total replacements: {total_replaced:,}")
    
    return df


### Step 1.5: Dataset Integration

* Integrate person, vehicle, and accident datasets into person-level data

* Join hierarchy: person (base) → vehicle → accident

* Overlapped columns are dropped

In [8]:
def join_person_vehicle_accident(
    person: pl.DataFrame,
    vehicle: pl.DataFrame,
    accident: pl.DataFrame
) -> pl.DataFrame:

    # Identify overlapping columns
    vehicle_cols = set(vehicle.columns)
    person_cols = set(person.columns)
    accident_cols = set(accident.columns)
    
    # Drop overlapping columns from vehicle (keep join keys)
    vehicle_person_overlap = (vehicle_cols & person_cols) - {"ST_CASE", "VEH_NO"}
    if vehicle_person_overlap:
        vehicle = vehicle.drop(list(vehicle_person_overlap))
    
    # Drop overlapping columns from accident (keep join keys)
    accident_person_overlap = (accident_cols & person_cols) - {"ST_CASE"}
    if accident_person_overlap:
        accident = accident.drop(list(accident_person_overlap))
    
    # Join person → vehicle
    per_veh = person.join(vehicle, on=["ST_CASE", "VEH_NO"], how="left")
    
    # Join person-vehicle → accident
    per_full = per_veh.join(accident, on="ST_CASE", how="left")
    
    return per_full


### Step 1.6: Data Quality Summary (optional)

Null value analysis

In [9]:
def print_data_summary(df: pl.DataFrame, title: str = "Data Summary"):

    # NULL analysis
    null_counts = {}
    for col in df.columns:
        null_count = df.filter(pl.col(col).is_null()).height
        if null_count > 0:
            null_pct = (null_count / df.shape[0]) * 100
            null_counts[col] = (null_count, null_pct)
    
    print(f"\nColumns with NULLs: {len(null_counts)}/{len(df.columns)}")


### Pipeline
Run the complete preprocessing step

In [10]:
def run_step1_pipeline(
    data_dir: Path,
    output_file: Optional[Path] = None,
    numeric_columns: Optional[List[str]] = None
) -> pl.DataFrame:
    
    # Check for numeric columns
    if numeric_columns is None:
        numeric_columns = NUMERIC_COLUMNS

    # 1.1: Load data
    accident = load_fars_csv("accident.csv", data_dir)
    vehicle = load_fars_csv("vehicle.csv", data_dir)
    person = load_fars_csv("person.csv", data_dir)
    
    # 1.2: Validate keys
    accident = remove_duplicate_keys(accident, ["ST_CASE"])
    vehicle = remove_duplicate_keys(vehicle, ["ST_CASE", "VEH_NO"])
    person = remove_duplicate_keys(person, ["ST_CASE", "VEH_NO", "PER_NO"])
    
    # 1.3: Convert to numeric
    accident = convert_to_numeric(accident, numeric_columns)
    vehicle = convert_to_numeric(vehicle, numeric_columns)
    person = convert_to_numeric(person, numeric_columns)
    
    # 1.4: Handle missing codes
    accident = replace_field_specific_missing_codes(accident)
    vehicle = replace_field_specific_missing_codes(vehicle)
    person = replace_field_specific_missing_codes(person)
    
    # 1.5: Integrate datasets
    df_integrated = join_person_vehicle_accident(person, vehicle, accident)
    
    # 1.6: Quality summary (optional)
    print_data_summary(df_integrated, "Integrated Person-Level Dataset")
    
    # 1.7: Save output
    if output_file is not None:
        output_path = Path(output_file)
        output_path.parent.mkdir(parents=True, exist_ok=True)

        # Save parquet
        df_integrated.write_parquet(output_path)
        print(f"Saved parquet to: {output_path}")

        # Save csv (optional, just for check) !!! EXTREMELY HEAVY
        # csv_path = output_path.with_suffix('.csv')
        # df_integrated.write_csv(csv_path)
        # print(f"Saved csv to: {csv_path}")
    
    print(f"Final dataset: {df_integrated.shape[0]:,} rows × {df_integrated.shape[1]} columns")
    
    return df_integrated

### Use

- The paths are set for use with a cloned repository
- Modify them as needed

In [12]:
# Configuration
DATA_DIR = Path("Dataset")
OUTPUT_FILE = Path("Dataset/St1_person_level_integrated.parquet")

# Run complete pipeline
df = run_step1_pipeline(
    data_dir=DATA_DIR,
    output_file=OUTPUT_FILE
)

Loaded: accident.csv
Shape: 37,654 rows × 80 columns
Loaded: vehicle.csv
Shape: 58,319 rows × 203 columns
Loaded: person.csv
Shape: 92,400 rows × 126 columns
No duplicates found
No duplicates found
No duplicates found
Converted 35 columns to numeric
Converted 20 columns to numeric
Converted 44 columns to numeric
 Replaced missing codes in 23 columns
 Total replacements: 160,816
 Replaced missing codes in 31 columns
 Total replacements: 524,756
 Replaced missing codes in 34 columns
 Total replacements: 630,350

Columns with NULLs: 237/337
Saved parquet to: Dataset\St1_person_level_integrated.parquet
Final dataset: 92,400 rows × 337 columns
